In [16]:
from dolfinx import default_scalar_type
from dolfinx.fem import (Constant, dirichletbc, Function, FunctionSpace, assemble_scalar,
                         form, locate_dofs_geometrical, locate_dofs_topological)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import XDMFFile, gmshio
from dolfinx.mesh import create_unit_square, locate_entities
from dolfinx.plot import vtk_mesh

from ufl import (SpatialCoordinate, TestFunction, TrialFunction,
                 dx, grad, inner)

from mpi4py import MPI

import meshio
import gmsh
import numpy as np
import pyvista
##########################################################################################################3

  

In [22]:
mesh, cell_markers, facet_markers = gmshio.read_from_msh("helix.msh", MPI.COMM_WORLD, gdim=3)

Info    : Reading 'helix.msh'...
Info    : 7982 nodes
Info    : 51264 elements
Info    : Done reading 'helix.msh'


In [31]:
with XDMFFile(MPI.COMM_WORLD, "helix.xdmf", "r") as xdmf:
    mesh = xdmf.read_mesh(name="In")
    ct = xdmf.read_meshtags(mesh, name="In")
mesh.topology.create_connectivity(mesh.topology.dim, mesh.topology.dim - 1)
with XDMFFile(MPI.COMM_WORLD, "mt.xdmf", "r") as xdmf:
    ft = xdmf.read_meshtags(mesh, name="Grid")

RuntimeError: <Grid> with name 'In' not found.